!jupyter-nbconvert SQLAlchemy_for_chemists.ipynb --to slides --post serve

Introduction to SQLAlchemy and Jupyter Notebooks
==========================================

### John Hey

#### Adapted from material by:
- Lee Bergstrand
- Matt McInnes

**Note:** This tutorial is based off the [official SQLAlchemy ORM tutorial](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html). 

Introduction to SQLAlchemy for Chemists
==========================

Firstly we need to import sqlalchemy and begin creating our database:

In [1]:
import sqlalchemy

# Connect to the database...
from sqlalchemy import create_engine

# We normally set echo=False, but here we want to see the sql we are produce later
engine = create_engine('sqlite:///:memory:', echo=True)

**Note** that the string ``` 'sqlite:///:memory:' ``` sets the backend to sqlite and creates a database in memory. (RAM) 

This is useful for testing but we can change it to save to the harddisk later if we want our data to be permenant.

To set this to a file, we can just replace `:memory:` with a file path i.e.:

```python
file_path = "/home/users/john/my_super_secret_database.db"
engine = create_engine('sqlite:///'+file_path, echo=True)```

### Creating Database-Backed Classes
---

Our next step is to define our classes which will be mapped onto tables within our database. 

We do this by defining a class in the normal pythonic way but with some extra SQLAlchemy commands. 

SQLAlchemy will then use this class to define and create the relavent tables.  

In SQLAlchemy the ORM utilizes classes inheriting from the SQLAlchemy base class. 

**Note:** ORMs (Object Relational Mapper) are classes which allow you to refer to data stored in your database from another language.

Columns within a database table can usually only store scalar data (Ints, Strings etc.) an ORM allows you to map more complex datatypes onto these columns within your database. 

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In our example we are going to create a database of molecules and structural motifs

Classes define RDBMS (Relational DataBase Management Scheme) table attributes:

- We must define the `__tablename__` attribute
- Then we define some `Column` attributes
>- Columns have datatypes (Integer, Float, etc.)
>- Columns have constraints (Primary Key, Foreign Key, etc.)

In [3]:
from sqlalchemy import Column, Integer, String, PickleType

class Molecule(Base):
    __tablename__ = 'molecules'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    atoms = Column(PickleType)
    # Defines to_string() representation 
    def __repr__(self):
        return f"<Molecule(name={self.name}, atoms={self.atoms}, id={self.id})>"

SQLAlchemy will now set up your table for you...

In [4]:
Base.metadata.create_all(engine)

2018-12-16 13:08:40,189 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-16 13:08:40,191 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 13:08:40,193 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-16 13:08:40,195 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 13:08:40,197 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("molecules")
2018-12-16 13:08:40,199 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 13:08:40,202 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE molecules (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	atoms BLOB, 
	PRIMARY KEY (id)
)


2018-12-16 13:08:40,202 INFO sqlalchemy.engine.base.Engine ()
2018-12-16 13:08:40,204 INFO sqlalchemy.engine.base.Engine COMMIT


##### Issued SQL:

```SQL
CREATE TABLE molecules (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	atoms BLOB, 
	PRIMARY KEY (id)
)
```

This SQL will look familiar to those of you that have interacted with SQL databases by hand

This instructs the database to create a new table with columns:
- `id`    -- an integer that cannot have no value (`NOT NULL`)
- `name`  -- a character array (i.e. a string) 
- `atoms` -- a binary blob, i.e. some arbitary data. 

It also sets the `id` column up as the primary key for this table.
- So this table will be indexed by the `id` column

### Creating a Session

---

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

The `session` object is created by the `sqlalchemy.orm.sessionmaker` class and bound to our database's engine. 

This `session` is our interface to the database. 
- We use this to add/remove/fetch items from the database. 

We will see this in action in the next section.

### Adding, Updating and Commiting Objects
---

Once you have your tables created you can then start creating objects and mapping them onto the database.

In [6]:
phenol = Molecule(name="phenol", atoms=["C", "C", "C", "C", "C", "C", "H", "H", "H", "H", "H", "O", "H"])
session.add(phenol)
print(phenol)

<Molecule(name=phenol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=None)>


**Note:** No SQL has been printed here and `id=None` because we have not yet committed this object to the database

There are a couple of ways to commit your new object to the database:

- Call the `session.commit()` method, or;
- When you do a query SQLAlchemy, your uncommited changes are flushed to the db (within a transaction). SQLAlchemy then requeries the database in order to gain the value of the objects primary key column.

In [7]:
our_molecule = session.query(Molecule).filter_by(name='phenol').first()

2018-12-16 13:08:40,232 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-16 13:08:40,234 INFO sqlalchemy.engine.base.Engine INSERT INTO molecules (name, atoms) VALUES (?, ?)
2018-12-16 13:08:40,235 INFO sqlalchemy.engine.base.Engine ('phenol', <memory at 0x7f43f05f8408>)
2018-12-16 13:08:40,238 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name = ?
 LIMIT ? OFFSET ?
2018-12-16 13:08:40,239 INFO sqlalchemy.engine.base.Engine ('phenol', 1, 0)


##### Issued SQL:

*Insert (Flush)*

```SQL
INSERT INTO molecules (name, atoms) VALUES ('phenol', <memory at 0x7effc62f4408>)
```

*Query Select*

```SQL
SELECT molecules.id AS molecules_id, 
        molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name = 'phenol'
 LIMIT 1 OFFSET 0```

Since both of `phenol` and `our_molecule` have the same primary key they are mapped to the same object.

In [8]:
print(f"phenol = {phenol}")
print(f"our_molecule = {our_molecule}")

print(phenol is our_molecule)

phenol = <Molecule(name=phenol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
our_molecule = <Molecule(name=phenol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
True


Multiple objects can be added to the session and objects can be modified.

In [9]:
session.add_all([
        Molecule(name="water", atoms=["O", "H", "H"]),
        Molecule(name="CO2", atoms=["O", "O", "C"]),
        Molecule(name="benzene", atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']),])

# We can modify the name if we decide that we want the systematic name instead
phenol.name = "Cyclohexa-1,3,5-trienol"  

Finally, the transaction can be commited. The remaining changes are flushed to the database.

In [10]:
session.commit()

2018-12-16 13:08:40,265 INFO sqlalchemy.engine.base.Engine UPDATE molecules SET name=? WHERE molecules.id = ?
2018-12-16 13:08:40,266 INFO sqlalchemy.engine.base.Engine ('Cyclohexa-1,3,5-trienol', 1)
2018-12-16 13:08:40,267 INFO sqlalchemy.engine.base.Engine INSERT INTO molecules (name, atoms) VALUES (?, ?)
2018-12-16 13:08:40,267 INFO sqlalchemy.engine.base.Engine ('water', <memory at 0x7f43f05f8408>)
2018-12-16 13:08:40,268 INFO sqlalchemy.engine.base.Engine INSERT INTO molecules (name, atoms) VALUES (?, ?)
2018-12-16 13:08:40,270 INFO sqlalchemy.engine.base.Engine ('CO2', <memory at 0x7f43f05f8588>)
2018-12-16 13:08:40,270 INFO sqlalchemy.engine.base.Engine INSERT INTO molecules (name, atoms) VALUES (?, ?)
2018-12-16 13:08:40,271 INFO sqlalchemy.engine.base.Engine ('benzene', <memory at 0x7f43f05f8408>)
2018-12-16 13:08:40,272 INFO sqlalchemy.engine.base.Engine COMMIT


##### Issued SQL:

*Flush remaining changes (inserts and updates)*

```SQL
UPDATE molecules SET name='Cyclohexa-1,3,5-trienol' WHERE molecules.id = 1

INSERT INTO molecules (name, atoms) VALUES ('water', <memory at 0x7fefdf101408>)
INSERT INTO molecules (name, atoms) VALUES ('CO2', <memory at 0x7fefdf101588>)
INSERT INTO molecules (name, atoms) VALUES ('benzene', <memory at 0x7fefdf101408>)
```

*Finally, commit the changes*

```SQL
COMMIT
```

### Rolling Back
---

In SQLAlchemy all database operations are performed within a transaction. 

For example we can edit and query for objects:

In [11]:
phenol.name = 'Phenol'

fake_molecule = Molecule(name='fakemol', atoms=["F", "Al", "Se"])
session.add(fake_molecule)

session.query(Molecule).filter(Molecule.name.in_(['Phenol', 'fakemol'])).all()

2018-12-16 13:08:40,279 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-16 13:08:40,282 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.id = ?
2018-12-16 13:08:40,283 INFO sqlalchemy.engine.base.Engine (1,)
2018-12-16 13:08:40,284 INFO sqlalchemy.engine.base.Engine UPDATE molecules SET name=? WHERE molecules.id = ?
2018-12-16 13:08:40,300 INFO sqlalchemy.engine.base.Engine ('Phenol', 1)
2018-12-16 13:08:40,302 INFO sqlalchemy.engine.base.Engine INSERT INTO molecules (name, atoms) VALUES (?, ?)
2018-12-16 13:08:40,303 INFO sqlalchemy.engine.base.Engine ('fakemol', <memory at 0x7f43f05f8408>)
2018-12-16 13:08:40,306 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name IN (?, ?)
2018-12-16 13:08:40,306 INFO sqlalchemy.engine.base.Engine ('Phenol', 'fakemol')


[<Molecule(name=Phenol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>,
 <Molecule(name=fakemol, atoms=['F', 'Al', 'Se'], id=5)>]

We can do a rollback and our changes are reset.

In [12]:
session.rollback()

2018-12-16 13:08:40,321 INFO sqlalchemy.engine.base.Engine ROLLBACK


##### Issued SQL:

```SQL
ROLLBACK
```

Our objects are reset to their pervious states and our uncommited objects are removed.

In [13]:
phenol.name

2018-12-16 13:08:40,331 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-12-16 13:08:40,332 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.id = ?
2018-12-16 13:08:40,333 INFO sqlalchemy.engine.base.Engine (1,)


'Cyclohexa-1,3,5-trienol'

In [14]:
fake_molecule in session

False

So our fake molecule is no longer part of our database session

### Querying
---

We can query our database for objects in the `Molecules` table and sort them

In [15]:
for instance in session.query(Molecule).order_by(Molecule.id):
    print(instance)

2018-12-16 13:08:40,352 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules ORDER BY molecules.id
2018-12-16 13:08:40,353 INFO sqlalchemy.engine.base.Engine ()
<Molecule(name=Cyclohexa-1,3,5-trienol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>
<Molecule(name=CO2, atoms=['O', 'O', 'C'], id=3)>
<Molecule(name=benzene, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'], id=4)>


##### Issued SQL:

```SQL
SELECT molecules.id AS molecules_id, 
        molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules ORDER BY molecules.id
```

**Note:** This returns all the information associated with each Molecule object.

Sometimes we only want to get certain information from each row:

In [16]:
for name, atoms in session.query(Molecule.name, Molecule.atoms):
    print(name, atoms)

2018-12-16 13:08:40,362 INFO sqlalchemy.engine.base.Engine SELECT molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules
2018-12-16 13:08:40,363 INFO sqlalchemy.engine.base.Engine ()
Cyclohexa-1,3,5-trienol ['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H']
water ['O', 'H', 'H']
CO2 ['O', 'O', 'C']
benzene ['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']


##### Issued SQL:

```SQL
SELECT molecules.name AS molecules_name, 
        molecules.atoms AS molecules_atoms 
FROM molecules
```

### Common Filtering Operators

We can filter our query so that we only select some of the data

Here’s an overview of some common operators used by the `filter()` fucntion:
- equals
- not equals
- like
- in
- not in
- and
- or

#### equals:

In [17]:
for molecule in session.query(Molecule).filter(Molecule.name == 'water'):
    print(molecule)

2018-12-16 13:08:40,372 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name = ?
2018-12-16 13:08:40,373 INFO sqlalchemy.engine.base.Engine ('water',)
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>


#### not equals:

In [18]:
for molecule in session.query(Molecule).filter(Molecule.name != 'water'):
    print(molecule)

2018-12-16 13:08:40,387 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name != ?
2018-12-16 13:08:40,391 INFO sqlalchemy.engine.base.Engine ('water',)
<Molecule(name=Cyclohexa-1,3,5-trienol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
<Molecule(name=CO2, atoms=['O', 'O', 'C'], id=3)>
<Molecule(name=benzene, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'], id=4)>


#### LIKE:

In [19]:
# everything that matches wat*
for molecule in session.query(Molecule).filter(Molecule.name.like('wat%')):
    print(molecule)   

2018-12-16 13:08:40,403 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name LIKE ?
2018-12-16 13:08:40,404 INFO sqlalchemy.engine.base.Engine ('wat%',)
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>


In [20]:
# everything with an "o" in the name:
for molecule in session.query(Molecule).filter(Molecule.name.like('%o%')):
    print(molecule)

2018-12-16 13:08:40,413 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name LIKE ?
2018-12-16 13:08:40,414 INFO sqlalchemy.engine.base.Engine ('%o%',)
<Molecule(name=Cyclohexa-1,3,5-trienol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
<Molecule(name=CO2, atoms=['O', 'O', 'C'], id=3)>


#### IN:

In [21]:
for molecule in session.query(Molecule).filter(Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water'])):
    print(molecule)

2018-12-16 13:08:40,426 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name IN (?, ?)
2018-12-16 13:08:40,427 INFO sqlalchemy.engine.base.Engine ('Cyclohexa-1,3,5-trienol', 'water')
<Molecule(name=Cyclohexa-1,3,5-trienol, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'O', 'H'], id=1)>
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>


#### NOT IN:

In [22]:
# use "tilda~" for NOT IN
for molecule in session.query(Molecule).filter(~Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water'])):
    print(molecule)

2018-12-16 13:08:40,437 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name NOT IN (?, ?)
2018-12-16 13:08:40,438 INFO sqlalchemy.engine.base.Engine ('Cyclohexa-1,3,5-trienol', 'water')
<Molecule(name=CO2, atoms=['O', 'O', 'C'], id=3)>
<Molecule(name=benzene, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'], id=4)>


#### AND:

In [23]:
for molecule in session.query(Molecule).filter(Molecule.name.in_(['Cyclohexa-1,3,5-trienol',
                                                                 'water']),
                                              Molecule.atoms == ["O", "H", "H"]):
    print(molecule)

2018-12-16 13:08:40,448 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name IN (?, ?) AND molecules.atoms = ?
2018-12-16 13:08:40,449 INFO sqlalchemy.engine.base.Engine ('Cyclohexa-1,3,5-trienol', 'water', <memory at 0x7f43f05f8588>)
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>


#### OR:

In [24]:
from sqlalchemy import or_
for molecule in session.query(Molecule).filter(or_(Molecule.name == "water", Molecule.id >= 4)):
    print(molecule)

2018-12-16 13:08:40,468 INFO sqlalchemy.engine.base.Engine SELECT molecules.id AS molecules_id, molecules.name AS molecules_name, molecules.atoms AS molecules_atoms 
FROM molecules 
WHERE molecules.name = ? OR molecules.id >= ?
2018-12-16 13:08:40,470 INFO sqlalchemy.engine.base.Engine ('water', 4)
<Molecule(name=water, atoms=['O', 'H', 'H'], id=2)>
<Molecule(name=benzene, atoms=['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'], id=4)>


### Relationships
---

This is where the real magic starts! 

Multiple classes can be created and linked together using class relations.

### Many to Many: 

We can link one motif to many molecules and vice versa: 

In [25]:
import sqlalchemy

# Connect to the database...
from sqlalchemy import create_engine

# We normally set echo=False, but here we want to see the sql we are produce later
engine = create_engine('sqlite:///:memory:', echo=False)

from sqlalchemy import Column, Integer, String, PickleType
from sqlalchemy import ForeignKey, Table

from sqlalchemy.orm import relationship, backref

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

We create our database tables as normal, with the addition that we define relationships for each one, pointing at the other: 

In [26]:
class Molecule(Base):
    __tablename__ = 'molecules'

    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    atoms = Column(PickleType)

    # ... any other fields

    functional_groups = relationship("Motif", secondary="mol_motif_assoc")

class Motif(Base):
    __tablename__ = 'motifs'

    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    atoms = Column(PickleType)

    appears_in = relationship("Molecule", secondary="mol_motif_assoc")

Here we must create an intermediate table which has columns containing the foreign keys of the two tables to be joined as well as relationships with both tables to be linked:

In [27]:
class MolMotifAssoc(Base):
    __tablename__ = 'mol_motif_assoc'
    
    id = Column(Integer, primary_key=True)
    
    molecule_id = Column(Integer, ForeignKey('molecules.id'))
    funtional_group_id = Column(Integer, ForeignKey('motifs.id'))

    molecule = relationship(
        Molecule, 
        backref=backref(
            "mol_motif_assoc", 
            cascade="all, delete-orphan"
        ))
    
    motif = relationship(
        Motif, 
        backref=backref(
            "mol_motif_assoc", 
            cascade="all, delete-orphan"
        ))

In [28]:
Base.metadata.create_all(engine)

##### Issued SQL:

```SQL

PRAGMA table_info("molecules")

PRAGMA table_info("motifs")

PRAGMA table_info("mol_motif_assoc")

CREATE TABLE molecules (
	id INTEGER NOT NULL, 
	name VARCHAR(255), 
	atoms BLOB, 
	PRIMARY KEY (id)
)

CREATE TABLE motifs (
	id INTEGER NOT NULL, 
	name VARCHAR(255), 
	atoms BLOB, 
	PRIMARY KEY (id)
)

CREATE TABLE mol_motif_assoc (
	id INTEGER NOT NULL, 
	molecule_id INTEGER, 
	funtional_group_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(molecule_id) REFERENCES molecules (id), 
	FOREIGN KEY(funtional_group_id) REFERENCES motifs (id)
)

COMMIT
```

In [29]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

session.commit()

Now we can start to create some molecules:

In [30]:
phenol = Molecule(name="Phenol", atoms=["C"]*6+["H"]*6+["O"])
TMAC = Molecule(name="TetraMethylAmmonimumChloride", atoms=["N"]+["H"]*4+["Cl"])
benzenediol = Molecule(name="Benzenediol", atoms=["C"]*6+["H"]*6+["O"]*2)
ethanol = Molecule(name="Ethanol", atoms=["C"]*2+["H"]*6+["O"])

And some functional groups:

In [31]:
phenyl = Motif(name="phenyl", atoms=["C"]*6)
alcohol = Motif(name="alcohol", atoms=["O"]+["H"])

Here we add all the new molecules and functional groups to the database:

In [32]:
session.add_all([phenyl, alcohol, phenol, TMAC, benzenediol, ethanol])
session.commit()

##### Issued SQL:

```SQL
INSERT INTO molecules (name, atoms) VALUES ('Phenol', <memory at 0x7f12f1bf9348>)

INSERT INTO molecules (name, atoms) VALUES ('TetraMethylAmmonimumChloride', <memory at 0x7f12f1bf9408>)

INSERT INTO molecules (name, atoms) VALUES ('Benzenediol', <memory at 0x7f12f1bf9348>)

INSERT INTO molecules (name, atoms) VALUES ('Ethanol', <memory at 0x7f12f1bf9408>)

INSERT INTO motifs (name, atoms) VALUES ('phenyl', <memory at 0x7f12f1bf9408>)

INSERT INTO motifs (name, atoms) VALUES ('alcohol', <memory at 0x7f12f1bf9348>)

COMMIT

```

Here we assign some functional groups to our molecules and commit this:

In [33]:
phenol.functional_groups.extend([phenyl, alcohol])
benzenediol.functional_groups.extend([phenyl, alcohol])
ethanol.functional_groups.append(alcohol)
session.commit()

Now we can query our molecule instances and see the  

In [34]:
print([fg.atoms for fg in phenol.functional_groups])
print(phenol.functional_groups[0].name)
print(phenyl.appears_in[0].name)
print(phenyl.appears_in[0].name, phenyl.mol_motif_assoc[0].motif.name)

[['C', 'C', 'C', 'C', 'C', 'C'], ['O', 'H']]
phenyl
Phenol
Phenol phenyl


We can also query the database for molecules that contain specific functional groups:

In [35]:
for molecule in session.query(Molecule).filter(Molecule.functional_groups.contains(alcohol)):
    print(molecule.name)

Phenol
Benzenediol
Ethanol


Or we can query for motifs contained within a molecule:

In [36]:
for fg in session.query(Motif).filter(Motif.appears_in.contains(benzenediol)):
    print(fg.name)

phenyl
alcohol


Or we could look for molecules that contain similar functional groups:

In [72]:
f_gs = session.query(Motif).filter(Motif.appears_in.contains(ethanol))
print([f.name for f in f_gs])

['alcohol']


In [71]:
for mol in session.query(Molecule).join(Molecule.functional_groups).filter(Motif.id.in_(fg.id for fg in f_gs)):
    print(mol.name)

Phenol
Benzenediol
Ethanol
